In [1]:
from IPython.display import Image

# 6. 모델 평과와 하이퍼파라미터 튜닝

# 6.1 파이프라인을 사용한 효율적인 워크플로
* 사이킷런의 Pipeline 클래스

### 6.1.1 위스콘신 유방암 데이터셋

In [2]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)
print(df.shape)
df.head()

(569, 32)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


* 악성과 양성인 종양 세포 샘플 569개
    * 0 : 고유 ID 번호
    * 1 : 진단 결과(M=악성, B=양성)
    * 3~32 : 세포 핵의 디지털 이미지에서 계산된 30개의 실수 값 특성

In [3]:
from sklearn.preprocessing import LabelEncoder

X = df.loc[:, 2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

클래스 레이블(진단 결과)을 배열 y에 인코딩하면 악성 종양은 클래스1로 표현되고 양성 종양은 클래스 0으로 표현된다.

In [4]:
le.transform(['M', 'B'])

array([1, 0])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

### 6.1.2 파이프라인으로 변환기와 추정기 연결
* 주성분 분석을 통해 초기 차원 30차원에서 좀 더 낮은 2차원 부분 공간으로 데이터를 압축.
* 선형 분류기에 주입 전, 특성 표준화

훈련 데이터셋과 테스트 데이터셋을 각각 학습하고 변환하는 단계를 구성하는 대신 StandardScale, PCA, LogisticRegression 객체를 하나의 파이프라인 연결.

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=2), LogisticRegression(random_state=1))
pipe_lr.fit(X_train, y_train)
print('테스트 정확도: %.3f' % pipe_lr.score(X_test, y_test))

테스트 정확도: 0.956


In [7]:
# 사이킷런의 파이프라인 작동 방식
Image(url='https://git.io/JtsTr', width=500) 

In [8]:
# 사이킷런의 추정기 객체 시각화
import matplotlib.pyplot as plt
from sklearn import set_config
set_config(display='diagram')
pipe_lr

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])

# 6.2 k-겹 교차 검증을 사용한 모델 성능 평가
* 교차 검증 기법
    * 홀드아웃 교차 검증(holdout cross-validation)
    * k-겹 교차 검증(k-fold cross-validation)

### 6.2.1 홀드아웃 방법
* 훈련 데이터셋, 검증 데이터셋, 테스트 데이터셋 세 개의 부분으로 나눈다.
    * 훈련 데이터셋 : 여러가지 모델을 훈련하는 데 사용
    * 검증 데이터셋 : 모델 선택에 사용
* 새로운 데이터에 대한 일반화 능력을 덜 편향되게 추정할 수 있다는 장점이 있다.


홀드아웃 방법은 훈련 데이터를 훈련 데이터셋과 검증 데이터셋으로 나누는 방법에 따라 성능 추정이 민감할 수 있다는 것이 단점이다.

### 6.2.2 k-겹 교차 검증
1. k-겹 교차 검증에서는 중복을 허용하지 않고 훈련 데이터셋을 k개의 폴드로 랜덤하게 나눈다.
2. k-1 개의 폴드로 모델을 훈련하고 나머지 하나의 폴드로 성능 평가.
3. 1, 2 과정을 k번 반복하여 k개의 모델과 성능 추청을 얻는다.
4. 서로 다른 독립적인 폴드에서 얻은 성능 추정을 기반으로 모델 평균 성능을 계산한다.


In [9]:
# 홀드아웃 교차 검증
Image(url='https://git.io/JtsT6', width=500) 

* LOOCV 방법
    * Leave-One-Out Cross-Validation
    * 폴드 개수가 훈련 샘플 개수와 같다. (k=n)
    * 하나의 훈련 샘플이 각 반복에서 테스트로 사용
    * 아주 작은 데이터셋을 사용할 때 권장

기본 k-겹 교차 검증 방법보다 좀 더 향상된 방법은 계층적 k-겹 교차 검증이다. 좀 더 나은 편향과 분산 추정을 만든다.

* 계층적 교차 검증
    * 각 폴드에서 클래스 비율이 전체 훈련 데이터셋에 클래스 비율을 대표하도록 유지
    * 사이킷런의 StratifiedKFold
        * shuffle 매개 변수를 True로 지정하면 폴드를 나누기 전에 샘플을 섞는다. default=False
        * shuffle=False일 때 random_state 매개변수를 지정하면 경고 발생.
        

In [10]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('폴드: %2d, 클래스 분포: %s, 정확도: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV 정확도: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

폴드:  1, 클래스 분포: [256 153], 정확도: 0.935
폴드:  2, 클래스 분포: [256 153], 정확도: 0.935
폴드:  3, 클래스 분포: [256 153], 정확도: 0.957
폴드:  4, 클래스 분포: [256 153], 정확도: 0.957
폴드:  5, 클래스 분포: [256 153], 정확도: 0.935
폴드:  6, 클래스 분포: [257 153], 정확도: 0.956
폴드:  7, 클래스 분포: [257 153], 정확도: 0.978
폴드:  8, 클래스 분포: [257 153], 정확도: 0.933
폴드:  9, 클래스 분포: [257 153], 정확도: 0.956
폴드: 10, 클래스 분포: [257 153], 정확도: 0.956

CV 정확도: 0.950 +/- 0.014


sklearn.model_selection 모듈에 있는 StratifiedKFold 클래스를 훈련 데이터셋의 y_train 클래스 레이블을 전달하여 초기화.

* n_splits : 폴드 개수 지정

In [11]:
# 계층별 k-겹 교차 검증을 사용하여 모델 평가
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=1)
print('CV 정확도 점수: %s' % scores)
print('CV 정확도: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV 정확도 점수: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도: 0.950 +/- 0.014


In [12]:
# 교차 검증에 여러 측정 지표를 사용할 수 있는 cross_validate
from sklearn.model_selection import cross_validate

scores = cross_validate(estimator=pipe_lr, X=X_train, y=y_train, scoring=['accuracy'], cv=10, n_jobs=-1)
print('CV 정확도 점수: %s' % scores['test_accuracy'])
print('CV 정확도: %.3f +/- %.3f' % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy'])))

CV 정확도 점수: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도: 0.950 +/- 0.014


`cross_val_predict` 함수는 `cross_val_score`와 비슷한 인터페이스를 제공하지만 훈련 데이터셋의 각 샘플이 테스트 폴드가 되었을 때 만들어진 예측을 반환. 따라서 cross_val_predict 함수의 결과를 사용하여 모델의 성능(정확도)을 계산하면 cross_val_score 함수의 결과와 다르며 바람직한 일반화 성능 추청이 아니다.

In [13]:
from sklearn.model_selection import cross_val_predict

preds = cross_val_predict(estimator=pipe_lr, X=X_train, y=y_train, cv=10, n_jobs=-1)
preds[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

`method` 매개변수에 반환될 값을 계산하기 위한 모델의 메서드를 지정할 수 있다.
* method='predict_proba' : 예측 확률 반환
* 'predict_proba', 'predict', 'predict_log_proba', 'decision_function' 등
* default='predict'

In [14]:
from sklearn.model_selection import cross_val_predict

preds = cross_val_predict(estimator=pipe_lr, X=X_train, y=y_train, method='predict_proba', n_jobs=-1)
preds[:10]

array([[9.94155991e-01, 5.84400909e-03],
       [7.31384063e-01, 2.68615937e-01],
       [9.69885576e-01, 3.01144241e-02],
       [8.40624737e-01, 1.59375263e-01],
       [9.97603229e-01, 2.39677100e-03],
       [9.99781102e-01, 2.18898446e-04],
       [9.99162903e-01, 8.37097403e-04],
       [1.22472302e-06, 9.99998775e-01],
       [1.04992527e-01, 8.95007473e-01],
       [3.78721109e-04, 9.99621279e-01]])